In [2]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [3]:
import numpy as np
import tflearn
import tensorflow
import random as rd
import json

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [4]:
with open("intents bot1.json") as f:
	dat = json.load(f)
	intents = dat['intents']
intents

[{'tag': 'greeting',
  'patterns': ['Hi',
   'How are you',
   'Is anyone there?',
   'Hello',
   'Good day',
   'Whats up'],
  'responses': ['Hello!',
   'Good to see you again!',
   'Hi there, how can I help?'],
  'context_set': ''},
 {'tag': 'goodbye',
  'patterns': ['cya',
   'See you later',
   'Goodbye',
   'I am Leaving',
   'Have a Good day'],
  'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'],
  'context_set': ''},
 {'tag': 'age',
  'patterns': ['how old',
   'how old is tim',
   'what is your age',
   'how old are you',
   'age?'],
  'responses': ['I am 18 years old!', '18 years young!'],
  'context_set': ''},
 {'tag': 'name',
  'patterns': ['what is your name',
   'what should I call you',
   'whats your name?'],
  'responses': ['You can call me Tim.',
   "I'm Tim!",
   "I'm Tim aka Tech With Tim."],
  'context_set': ''},
 {'tag': 'shop',
  'patterns': ['Id like to buy something',
   'whats on the menu',
   'what do you reccommend?',
   'could i get som

In [5]:
class doc_struct:
	def __init__(self, tag, words):
		self.tag = tag
		self.words = words

words = []
labels = [intent["tag"] for intent in intents]
labels = list(set(labels))
docs = []

for intent in intents:
	for pattern in intent['patterns']:
		wrds = nltk.word_tokenize(pattern)
		words.extend(wrds) # concatenate 2 list
		docs.append(doc_struct(intent["tag"], wrds))
		

In [6]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"] # stemmer: extract root word 
words = sorted(list(set(words))) # remove duplicates
labels = sorted(labels)

In [7]:
'''
Representation: bag of words
for example, wrds= ['a', 'apple', 'eat', 'you', 'i']
sent = ['a', 'apple', 'i']
bag(sent, wrds) = [1, 1, 0, 0, 1]
whose 1 denotes existence of corresponded word, 0 is not.
'''
training = []
output = []
out_empty = [0 for _ in range(len(labels))]

for doc in docs:
	bag = []
	wrds = [stemmer.stem(w) for w in doc.words]
	for w in words:
		if w in wrds:
			bag.append(1)
		else:
			bag.append(0)
	output_row = out_empty[:]
	output_row[labels.index(doc.tag)] = 1

	training.append(bag)
	output.append(output_row)

In [8]:
training = np.array(training)
output = np.array(output)

In [9]:
# Buidling AI
tensorflow.compat.v1.reset_default_graph() # this is required to run afterwards
net = tflearn.input_data(shape=[None, len(training[0])])

net = tflearn.fully_connected(net, 8) # 8 neurons
net = tflearn.fully_connected(net, 8) # 8 neurons
net = tflearn.fully_connected(net, len(output[0]), activation="softmax") # output layers
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
# model.fit(training, output, n_epoch=1000, batch_size = 8, show_metric = True)
# model.save("model.tflearn")

# model.load("model.tflearn")

In [11]:
def bag_of_words(sent, words):
	bag = [0 for _ in range(len(words))]
	sent_wrds = nltk.word_tokenize(sent)
	sent_wrds = [stemmer.stem(w.lower()) for w in sent_wrds]

	# counting freq for each word
	for word in sent_wrds:
		for i, w in enumerate(words):
			if w == word:
				bag[i] = 1
	return np.array(bag)


In [12]:
def chat():
	print("You're talking with the bot (type quit to quit)!")

	while True:
		user_inp = input("You: ")
		if user_inp.lower() == "quit":
			break

		results = model.predict([bag_of_words(user_inp, words)])[0]
		results_idx = np.argmax(results)
		tag = labels[results_idx]
		if results[results_idx] > 0.7:

			for corspd in dat['intents']:
				if corspd['tag'] == tag:
					responses = corspd['responses']
			
			print(rd.choice(responses))
		else:
			print("I didn't get that, try again")
# chat()

In [13]:
with open("cafe.json") as f:
	corspd= json.load(f)
patterns = [inst['patterns'] for  inst in corspd]
responses = [inst['responses'] for  inst in corspd]
words = []
for pattern in patterns:
	wrds = nltk.word_tokenize(pattern)
	words.extend(wrds) # concatenate 2 list
words = list(set(words))
words = [w for w in words if w not in '?&,.']
words = sorted(words)
words

['11:00',
 '2',
 '2-4-20',
 '3',
 '5',
 'Americano',
 'Apple',
 'Asparagus',
 'Au',
 'Avocado',
 'Banana',
 'Basque',
 'Beef',
 'Black',
 'Blaeter',
 'Blueberry',
 'Bluree',
 'Cappucchino',
 'Cappuchino',
 'Caramel',
 'Chans',
 'Char',
 'Cheery',
 'Cheese',
 'Cherry',
 'Choco',
 'Chocolate',
 'Cinnimon',
 'Coffee',
 'Combo',
 'Con',
 'Cotton',
 'Cranberry',
 'Cream',
 'Cupcake',
 'Do',
 'Drumstick',
 'Easy',
 'Egg',
 'Espresso',
 'Forest',
 'Frappe',
 'French',
 'Fruit',
 'Honey',
 'I',
 'Ice-cream',
 'Iced',
 'Irish',
 'Keto',
 'Laite',
 'Late',
 'Latte',
 'Layer',
 'Macchiato',
 'Macrons',
 'Malted',
 'Mango',
 'Maxres',
 'Milkshake',
 'Mint',
 'Mix',
 'Mose',
 'Nutella',
 'Orange',
 'Oreo',
 'Panna',
 'Pastery',
 'Pastry',
 'Peanut',
 'Pineapple',
 'Pista',
 'Pretze',
 'Pretzel',
 'Puff',
 'Pumpkin',
 'Question',
 'Red',
 'Rough',
 'Salted',
 'Shake',
 'Shot',
 'Simpl',
 'Simple',
 'Slice',
 'Smores',
 'Smothie',
 'Social',
 'Special',
 'Spinach',
 'Strawberry',
 'Sui',
 'Tanuki',
 

In [14]:
training = []
output = []
out_empty = [0 for _ in range(len(corspd))]

for i, pair in enumerate(corspd):
	response = pair['responses'][0]
	wrds = nltk.word_tokenize(response)
	bag = []
	wrds = [stemmer.stem(w) for w in wrds]
	for w in words:
		if w in wrds:
			bag.append(1)
		else:
			bag.append(0)
	output_row = out_empty[:]
	output_row[i] = 1

	training.append(bag)
	output.append(output_row)
training = np.array(training)
output = np.array(output)

In [17]:
# Buidling AI
tensorflow.compat.v1.reset_default_graph() # this is required to run afterwards
net = tflearn.input_data(shape=[None, len(training[0])])

net = tflearn.fully_connected(net, 8) # 8 neurons
net = tflearn.fully_connected(net, 8) # 8 neurons
net = tflearn.fully_connected(net, len(output[0]), activation="softmax") # output layers
net = tflearn.regression(net)

model2 = tflearn.DNN(net)

In [18]:
model2.load("model2.tflearn")

INFO:tensorflow:Restoring parameters from c:\Users\Leong Teng Man\Documents\GitHub\AI-School-Project\model2.tflearn


In [33]:
[inst['responses'] for  inst in corspd]

[['Hello! How may I help you.'],
 ['Yes sir  Simple Coffee ,Cappuchino, Americano, Au Laite ,Au Laite Cappucchino ,Chans Espresso ,Con Panna, Black Coffee, Espresso, Frappe ,French Coffee ,Iced Coffee Late ,Irish Coffee, Latte Macchiato and  Wainans Choco Coffee.What would u like to have?.What would u like to have?'],
 ['Sir thanks for your order. You have ordered 1 espresso and 5 americano.Please wait for sometime.'],
 [''],
 ['I may suggest you Pista Shake'],
 ['Yes we have your choice in our cafe.'],
 ['Yes we have your choice in our cafe.'],
 ['Yes we have your choice in our cafe.'],
 ['Pardon please, what you want ?'],
 ["hello how may i help you Ovidiu_Rom l'_Affaire l'_Affaire l'_Affaire O"],
 ['Pardon please, what you want ?'],
 ['Yes sir  Simple Coffee ,Cappuchino, Americano, Au Laite ,Au Laite Cappucchino ,Chans Espresso ,Con Panna, Black Coffee, Espresso, Frappe ,French Coffee ,Iced Coffee Late ,Irish Coffee, Latte Macchiato and  Wainans Choco Coffee.What would u like to hav

In [45]:
responses = [inst['responses'] for  inst in corspd]
[i for i, inst in enumerate(responses) if inst == ['']]
responses[3]

['']

In [46]:
def chat2():
	print("You're talking with the bot (type quit to quit)!")
	responses = [inst['responses'] for  inst in corspd]
	while True:
		user_inp = input("You: ")
		if user_inp.lower() == "quit":
			break

		results = model2.predict([bag_of_words(user_inp, words)])[0]
		
		results_idx = np.argmax(results)
		print(results_idx)
		print(responses[results_idx])
chat2()

You're talking with the bot (type quit to quit)!
16
['']
16
['']
16
['']
16
['']
16
['']
16
['']
